In [1]:
import pandas as pd
import re
import numpy as np

In [ ]:
# была поставлена задача - собрать данные, на основе которых можно будет построить модель прогноза
# рождаемости первых, вторых 3 и более детей
# перед началом работы тщательно изучал ряд статей и опросов, а также других исследований
# чтобы определить - какие факторы характеризуют рождаемость (это коэффициенты рождаемости)
# и чтобы составить пул факторов, которые могут влиять на рождаемость
# в работе использовал выгрузки с официальной статистикой с сайта ЕМИИС

In [2]:
# для начала выгрузил данные по рождаемости мужчин
df_popul_men = pd.read_excel('численность мужчин.xls', header=2)
df_popul_men = df_popul_men.rename(columns={'Unnamed: 0': 'year',
                                            'Unnamed: 1': 'region',
                                            'Unnamed: 2': 'age',
                                            'городское население': 'men_population_city',
                                            'сельское население': 'men_population_country'})
df_popul_men.head()

,year,region,age,men_population_city,men_population_country
0,2018,Российская Федерация,0,649689,215574
1,2018,Российская Федерация,0-4 года,3554849,1248044
2,2018,Российская Федерация,1 год,730880,234861
3,2018,Российская Федерация,10 лет,584949,245119
4,2018,Российская Федерация,100 и более лет,4135,608


In [3]:
# выгрузил данные о рождаемости женщин
df_popul_women = pd.read_excel('численность женщин.xls', header=2)
df_popul_women = df_popul_women.rename(columns={'Unnamed: 0': 'year',
                                                'Unnamed: 1': 'region',
                                                'Unnamed: 2': 'age',
                                                'городское население': 'women_population_city',
                                                'сельское население': 'women_population_country'})
df_popul_women.head()

,year,region,age,women_population_city,women_population_country
0,2018,Российская Федерация,0,614369,203798
1,2018,Российская Федерация,0-4 года,3364125,1180008
2,2018,Российская Федерация,1 год,692307,222010
3,2018,Российская Федерация,10 лет,555984,232796
4,2018,Российская Федерация,100 и более лет,10483,2354


In [4]:
# файлы с выгрузками выше имеют идентичную структуру - соединим их и проверим на наличие пропусков
df_popul = df_popul_men.merge(df_popul_women, how='left', on=['year', 'region', 'age'])
df_popul.info() # пропущенных, как видим, нет

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60334 entries, 0 to 60333
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   year                      60334 non-null  int64 
 1   region                    60334 non-null  object
 2   age                       60334 non-null  object
 3   men_population_city       60334 non-null  int64 
 4   men_population_country    60334 non-null  int64 
 5   women_population_city     60334 non-null  int64 
 6   women_population_country  60334 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 3.7+ MB


In [5]:
# оставим только года с 2018 по 2021
df_popul = df_popul.query('year in (2019, 2020, 2021)')

In [6]:
# посмотрим на уникальные значения в столбце region
df_popul.region.unique()

array(['Российская Федерация', '    Центральный федеральный округ',
       '        Белгородская область', '        Брянская область',
       '        Владимирская область', '        Воронежская область',
       '        Ивановская область', '        Калужская область',
       '        Костромская область', '        Курская область',
       '        Липецкая область', '        Московская область',
       '        Орловская область', '        Рязанская область',
       '        Смоленская область', '        Тамбовская область',
       '        Тверская область', '        Тульская область',
       '        Ярославская область',
       '        Город Москва столица Российской Федерации город федерального значения',
       '    Северо-Западный федеральный округ',
       '        Республика Карелия', '        Республика Коми',
       '        Архангельская область',
       '            Ненецкий автономный округ (Архангельская область)',
       '            Архангельская область (кроме Ненец

In [7]:
# отбрасываем строки с РФ и федеральными округами
df_popul = df_popul[df_popul.region.apply(lambda x: 'федеральный' not in x and x != 'Российская Федерация')] 

In [8]:
# смотрим, что у нас в столбце age (возрастные категории)
df_popul.age.unique()

array(['0', '0-4 года', '1 год', '10 лет', '100 и более лет', '10-14 лет',
       '11 лет', '12 лет', '13 лет', '14 лет', '15 лет', '15-19 лет',
       '16 лет', '17 лет', '18 лет', '19 лет', '2 года', '20 лет',
       '20-24', '21 год', '22 лет', '23 лет', '24 лет', '25 лет', '25-29',
       '26 лет', '27 лет', '28 лет', '29 лет', '3 года', '30 лет',
       '30-34', '31 лет', '32 лет', '33 лет', '34 лет', '35 лет', '35-39',
       '36 лет', '37 лет', '38 лет', '39 лет', '4 года', '40 лет',
       '40-44', '41 лет', '42 лет', '43 лет', '44 лет', '45 лет',
       '45-49 лет', '46 лет', '47 лет', '48 лет', '49 лет', '5 лет',
       '50 лет', '50-54 лет', '51 лет', '52 лет', '53 года', '54 лет',
       '55 лет', '55-59 лет', '56 лет', '57 лет', '58 лет', '59 лет',
       '5-9 лет', '6 лет', '60 лет', '60-64 лет', '61 лет', '62 года',
       '63 года', '64 года', '65 года', '65-69 лет', '66 года', '67 лет',
       '68 лет', '69 лет', '7 лет', '70 лет', '70-74 лет', '71 год',
       '72 год

In [9]:
# отбрасываем все нечисловые столбцы
df_popul = df_popul[df_popul.age.str.contains('^\d+ ', flags=re.IGNORECASE, regex=True)]
df_popul.age.unique()

array(['1 год', '10 лет', '100 и более лет', '11 лет', '12 лет', '13 лет',
       '14 лет', '15 лет', '16 лет', '17 лет', '18 лет', '19 лет',
       '2 года', '20 лет', '21 год', '22 лет', '23 лет', '24 лет',
       '25 лет', '26 лет', '27 лет', '28 лет', '29 лет', '3 года',
       '30 лет', '31 лет', '32 лет', '33 лет', '34 лет', '35 лет',
       '36 лет', '37 лет', '38 лет', '39 лет', '4 года', '40 лет',
       '41 лет', '42 лет', '43 лет', '44 лет', '45 лет', '46 лет',
       '47 лет', '48 лет', '49 лет', '5 лет', '50 лет', '51 лет',
       '52 лет', '53 года', '54 лет', '55 лет', '56 лет', '57 лет',
       '58 лет', '59 лет', '6 лет', '60 лет', '61 лет', '62 года',
       '63 года', '64 года', '65 года', '66 года', '67 лет', '68 лет',
       '69 лет', '7 лет', '70 лет', '71 год', '72 года', '73 года',
       '74 года', '75 лет', '76 лет', '77 лет', '78 лет', '79 лет',
       '8 лет', '80 лет', '81 год', '82 года', '83 года', '84 года',
       '85 лет', '86 лет', '87 лет', '88 лет',

In [10]:
# преобразуем в числовые значения, а затем в интервалы
df_popul.age = df_popul.age.apply(lambda x: int(x.strip(' лет и более года')))
bins = pd.IntervalIndex.from_tuples([(0, 17), (17, 20), (20, 25), (25, 30), (30, 35), (35, 40), (40, 100)])
df_popul.age = pd.cut(df_popul.age, bins)

# меняем интервалы на строки
df_popul.age = df_popul.age.apply(lambda x: str(x).replace(', ', '_').replace('(', '').replace(']', '').replace('17', '18'))

In [11]:
# меняем тип переменной и отбираем только нужные нам интервалы
df_popul = df_popul.astype({'age': np.object0}) \
    .query('age != "0_18"')

In [12]:
# добавим столбцы с совокупным женским и мужским населением
df_popul['men_population'] = df_popul.men_population_city + df_popul.men_population_country
df_popul['women_population'] = df_popul.women_population_city + df_popul.women_population_country
# смотрим на итоговый датафрейм
df_popul

,year,region,age,men_population_city,men_population_country,women_population_city,women_population_country,men_population,women_population
12970,2019,Белгородская область,40_100,20,12,55,87,32,142
12980,2019,Белгородская область,18_20,4597,2296,4600,1954,6893,6554
12981,2019,Белгородская область,18_20,3902,1940,4191,2005,5842,6196
12983,2019,Белгородская область,18_20,4015,2152,4488,2161,6167,6649
12985,2019,Белгородская область,20_25,4075,2301,4280,2157,6376,6437
...,...,...,...,...,...,...,...,...,...
48421,2021,Чукотский автономный округ,40_100,0,0,0,0,0,0
48423,2021,Чукотский автономный округ,40_100,1,0,0,0,1,0
48424,2021,Чукотский автономный округ,40_100,0,1,0,0,1,0
48425,2021,Чукотский автономный округ,40_100,0,0,1,0,0,1


In [13]:
# заранее сохраним наименования групп
age_groups = sorted(df_popul.age.unique())

# группируем, считаем население по возрастным группам
df_popul = df_popul.groupby(['year', 'region', 'age']) \
    .agg({i: 'sum' for i in df_popul.columns[3:]})

# перекидываем строки в столбцы
df_popul = df_popul.unstack()

# на текущем этапе удобнее всего переходить к относительным показателям - переходим к промилле
# могут возникнуть NaN значения, т.к. возможно нулевое население по году-региону
df_popul.men_population_city = (df_popul.men_population_city / df_popul.men_population * 1000).fillna(0)
df_popul.men_population_country = (df_popul.men_population_country / df_popul.men_population * 1000).fillna(0)
df_popul.women_population_city = (df_popul.women_population_city / df_popul.women_population * 1000).fillna(0)
df_popul.women_population_country = (df_popul.women_population_country / df_popul.women_population * 1000).fillna(0)

# отбрасываем столбцы с суммарным населением - больше они не нужны
df_popul = df_popul[['men_population_city', 'men_population_country', 'women_population_city', 'women_population_country']]

# далее мой способ избавиться от мультииндексов
# сохраняю в эксель
df_popul.sort_index(level=0, ascending=True).to_excel('test.xlsx')

df_popul = pd.read_excel('test.xlsx', header=2) # когда читаем сохраненный файл - режутся индексы + сразу со второй строки прочел

df_popul.year = sorted(i for _ in range(87) for i in (2019, 2020, 2021)) # переформировываю столбец с годами

# формирую список новых наименований столбцов
new_с = [f'{i}_population_{k}_{j}' 
         for i in ('men', 'women')
         for k in ('city', 'country')
         for j in age_groups]

# меняю наименования столбцов
df_popul = df_popul.rename(columns={k: n for k, n in zip(df_popul.columns[2:], new_с)}) 
df_popul # смотрим, что получилось

,year,region,men_population_city_18_20,men_population_city_20_25,men_population_city_25_30,men_population_city_30_35,men_population_city_35_40,men_population_city_40_100,men_population_country_18_20,men_population_country_20_25,...,women_population_city_25_30,women_population_city_30_35,women_population_city_35_40,women_population_city_40_100,women_population_country_18_20,women_population_country_20_25,women_population_country_25_30,women_population_country_30_35,women_population_country_35_40,women_population_country_40_100
0,2019,Архангельская область (кроме Ненец...,857.692065,841.760469,833.786332,812.975801,799.733128,721.455984,142.307935,158.239531,...,952.332595,894.373383,851.689936,757.758459,120.284431,144.241429,47.667405,105.626617,148.310064,242.241541
1,2019,Ненецкий автономный округ (Арханге...,666.246851,727.195946,723.703704,795.857988,775.467775,677.752583,333.753149,272.804054,...,859.154930,886.097152,834.675255,705.847136,284.591195,233.653846,140.845070,113.902848,165.324745,294.152864
2,2019,Тюменская область (кроме Ханты-Ман...,655.934161,669.366424,760.707787,739.095143,710.483912,620.277318,344.065839,330.633576,...,794.794388,749.931118,714.909849,644.363763,348.643006,337.845552,205.205612,250.068882,285.090151,355.636237
3,2019,Ханты-Мансийский автономный округ ...,929.107478,920.409626,923.369748,925.402500,925.747283,914.996244,70.892522,79.590374,...,938.418331,942.066448,939.731967,925.418435,70.034443,74.486952,61.581669,57.933552,60.268033,74.581565
4,2019,Ямало-Ненецкий автономный округ (Т...,789.708585,799.581477,841.317365,868.065485,873.170917,855.890603,210.291415,200.418523,...,861.854800,874.124069,872.145598,855.722345,211.731911,196.749933,138.145200,125.875931,127.854402,144.277655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,2021,Челябинская область,829.117968,797.103824,840.189987,841.228926,849.699354,808.672684,170.882032,202.896176,...,873.708255,875.959208,863.585691,829.145172,177.563013,193.811611,126.291745,124.040792,136.414309,170.854828
257,2021,Чеченская Республика,349.950364,354.663510,365.896759,422.155689,436.731398,363.960992,650.049636,645.336490,...,382.673192,413.988062,405.950875,360.165062,646.898803,638.404296,617.326808,586.011938,594.049125,639.834938
258,2021,Чувашская Республика - Чувашия,642.122761,557.749059,690.627202,662.072535,666.986521,541.795085,357.877239,442.250941,...,850.198545,815.393627,774.747475,604.036203,356.550283,400.074495,149.801455,184.606373,225.252525,395.963797
259,2021,Чукотский автономный округ,648.241206,806.693989,740.790355,753.108348,758.071107,737.025900,351.758794,193.306011,...,861.400894,718.819599,756.973995,720.754717,407.228916,123.209169,138.599106,281.180401,243.026005,279.245283


In [14]:
# займемся другой частью данных
# в файле представлены данные по числу родившихся первых, вторых, третьих, четвертых, пятых и более детей
# в разбивке по годам (2018, 2019, 2020, 2021)
# в разбивке по регионам (+ общее по РФ и по федеральным округам)
# далее идет детализация по возрасту женщин
df = pd.read_excel('Число_родившихся_детей.xls')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19968 entries, 0 to 19967
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  19968 non-null  int64 
 1   Unnamed: 1  19968 non-null  object
 2   Unnamed: 2  19968 non-null  object
 3   1           19968 non-null  int64 
 4   2           19968 non-null  int64 
 5   3           19968 non-null  int64 
 6   4           19968 non-null  int64 
 7   5 и более   19968 non-null  int64 
 8   Всего       19968 non-null  int64 
 9   Неизвестно  19968 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 1.5+ MB


In [15]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,1,2,3,4,5 и более,Всего,Неизвестно
0,2018,Российская Федерация,11 лет,0,0,0,0,0,0,0
1,2018,Российская Федерация,12 лет,4,0,0,0,0,4,0
2,2018,Российская Федерация,13 лет,34,0,0,0,0,34,0
3,2018,Российская Федерация,14 лет,190,4,0,0,0,195,1
4,2018,Российская Федерация,15 лет,894,22,0,0,0,919,3


In [16]:
# немного преобразуем наши данные 
# поменяем названия, отбросим строки и столбцы с итоговыми суммами и неизвестными параметрами
df = df.rename(columns={'Unnamed: 0': 'year', 
                        'Unnamed: 1': 'region',
                        'Unnamed: 2': 'woman_age',
                        '1': 'first',
                        '2': 'second',
                        '3': 'third',
                        '4': 'fourth',
                        '5 и более': 'five_more'})
df = df.drop(columns=['Неизвестно', 'Всего'])
df = df.query('woman_age != "Возраст неизвестен" and woman_age != "Всего" and region != "Российская Федерация"')

In [17]:
# пропусков нет - работаем дальше
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19000 entries, 52 to 19965
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year       19000 non-null  int64 
 1   region     19000 non-null  object
 2   woman_age  19000 non-null  object
 3   first      19000 non-null  int64 
 4   second     19000 non-null  int64 
 5   third      19000 non-null  int64 
 6   fourth     19000 non-null  int64 
 7   five_more  19000 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 1.3+ MB


In [18]:
# посмотрим, что у нас в колонке с регионом
df.region.unique()

array(['    Центральный федеральный округ',
       '        Белгородская область', '        Брянская область',
       '        Владимирская область', '        Воронежская область',
       '        Ивановская область', '        Калужская область',
       '        Костромская область', '        Курская область',
       '        Липецкая область', '        Московская область',
       '        Орловская область', '        Рязанская область',
       '        Смоленская область', '        Тамбовская область',
       '        Тверская область', '        Тульская область',
       '        Ярославская область',
       '        Город Москва столица Российской Федерации город федерального значения',
       '    Северо-Западный федеральный округ',
       '        Республика Карелия', '        Республика Коми',
       '        Архангельская область',
       '            Ненецкий автономный округ (Архангельская область)',
       '            Архангельская область (кроме Ненецкого автономного округа)

In [19]:
# убираем федеральные округа
df = df[df['region'].apply(lambda x: 'федеральный' not in x)] 

In [20]:
# переведем возраст женщин в числовые значения
# тут есть нюанс - 65 -имеется в виду 65 и более лет и мы теряем этот при переходе к int значениям
# с другой стороны - женщин, которые рожают в возрасте 65 лет и старше очень мало
# поэтому подобным нюансом можно пренебречь
df.woman_age = df.woman_age.apply(lambda x: int(x.strip(' лет и более года')))

In [21]:
# чисто для успокоения души - прикинем отношение случаев рождения в возрасте 65 и более лет от общего числа новорожденных
res = df.groupby('woman_age', as_index=False) \
    .agg({i: 'sum' for i in (df.columns[3:])})

for col in res.columns[1:]:
    res[col] = res[col].apply(lambda x: x / res[col].sum() * 100)
    
res.query('woman_age == 65')
# меньше 2 тысячных процента - это максимальное отклонение, которое возможно.
# очевидно, можно пренебречь такими данными

,woman_age,first,second,third,fourth,five_more
49,65,0.00031,0.000483,0.001204,0.001798,0.001084


In [22]:
# взглянем, что у нас получилось - у нас есть данные по году, региону, возрасту родивших, 
# а также информация по очередности рождения
df.head(5)

,year,region,woman_age,first,second,third,fourth,five_more
104,2018,Белгородская область,11,0,0,0,0,0
105,2018,Белгородская область,12,0,0,0,0,0
106,2018,Белгородская область,13,0,0,0,0,0
107,2018,Белгородская область,14,0,0,0,0,0
108,2018,Белгородская область,15,3,0,0,0,0


In [23]:
# преобразуем в числовые значения, а затем в интервалы
bins = pd.IntervalIndex.from_tuples([(10, 17), (17, 20), (20, 25), (25, 30), (30, 35), (35, 40), (40, 100)])
df.woman_age = pd.cut(df.woman_age, bins)

# меняем интервалы на строки
df.woman_age = df.woman_age.apply(lambda x: str(x).replace(', ', '_').replace('(', '').replace(']', '').replace('17', '18'))

# меняем тип переменной и отбираем только нужные нам интервалы
df = df.astype({'woman_age': np.object0}) \
    .query('woman_age != "10_18"')

In [24]:
# пересчитаем рождаемость по группам в зависимости от диапазонов возраста женщин
new_df = df.groupby(['year', 'region', 'woman_age']) \
    .agg({i: 'sum' for i in df.columns[3:]})
    
new_df.head(10)

first  \
year region                                             woman_age          
2018             Архангельская область (кроме Ненецк... 18_20        406   
                                                        20_25       1136   
                                                        25_30       1324   
                                                        30_35        654   
                                                        35_40        176   
                                                        40_100        29   
                 Ненецкий автономный округ (Архангел... 18_20         30   
                                                        20_25         57   
                                                        25_30         45   
                                                        30_35         28   

                                                                   second  \
year region                                             woman_age           
2018             Архангельская область (кроме Ненецк... 18_20          67   
                                                        20_25         533   
                                                        25_30        1711   
                                                        30_35        1517   
                                                        35_40         529   
                                                        40_100         73   
                 Ненецкий автономный округ (Архангел... 18_20          12   
                                                        20_25          37   
                                                        25_30          86   
                                                        30_35          70   

                                                                   third  \
year region                                             woman_age          
2018             Архангельская область (кроме Ненецк... 18_20          8   
                                                        20_25         85   
                                                        25_30        474   
                                                        30_35        715   
                                                        35_40        428   
                                                        40_100        89   
                 Ненецкий автономный округ (Архангел... 18_20          0   
                                                        20_25          8   
                                                        25_30         48   
                                                        30_35         41   

                                                                   fourth  \
year region                                             woman_age           
2018             Архангельская область (кроме Ненецк... 18_20           1   
                                                        20_25          16   
                                                        25_30          91   
                                                        30_35         139   
                                                        35_40         128   
                                                        40_100         23   
                 Ненецкий автономный округ (Архангел... 18_20           0   
                                                        20_25           3   
                                                        25_30          14   
                                                        30_35          23   

                                                                   five_more  
year region                                             woman_age             
2018             Архангельская область (кроме Ненецк... 18_20              0  
                                                        20_25              1  
                                                        25_30             4

In [25]:
# перекидываю индексы woman_age в столбцы
new_df = new_df.unstack()

# переходим к промилле
new_df['full_birth'] = new_df[['first', 'second', 'third', 'fourth', 'five_more']].sum(axis=1)
for col in new_df.columns:
    new_df[col] = (new_df[col] / new_df['full_birth'] * 1000).fillna(0)
    
# сразу отбрасываем столбец с общей суммой - больше он не нужен
new_df = new_df[['first', 'second', 'third', 'fourth', 'five_more']]

# тут получился, мягко говоря, неидеальный результат - пришлось поломать голову с мультииндексами
new_df.head()

first  \
woman_age                                                    18_20   
year region                                                          
2018             Архангельская область (кроме Ненецк...  38.443329   
                 Ненецкий автономный округ (Архангел...  49.342105   
                 Тюменская область (кроме Ханты-Манс...  38.942706   
                 Ханты-Мансийский автономный округ -...  36.028002   
                 Ямало-Ненецкий автономный округ (Тю...  44.425817   

                                                                     \
woman_age                                                     20_25   
year region                                                           
2018             Архангельская область (кроме Ненецк...  107.565571   
                 Ненецкий автономный округ (Архангел...   93.750000   
                 Тюменская область (кроме Ханты-Манс...  110.207857   
                 Ханты-Мансийский автономный округ -...  122.620056   
                 Ямало-Ненецкий автономный округ (Тю...  111.902766   

                                                                     \
woman_age                                                     25_30   
year region                                                           
2018             Архангельская область (кроме Ненецк...  125.366916   
                 Ненецкий автономный округ (Архангел...   74.013158   
                 Тюменская область (кроме Ханты-Манс...  123.545733   
                 Ханты-Мансийский автономный округ -...  118.473269   
                 Ямало-Ненецкий автономный округ (Тю...  105.476390   

                                                                               \
woman_age                                                    30_35      35_40   
year region                                                                     
2018             Архангельская область (кроме Ненецк...  61.925954  16.665089   
                 Ненецкий автономный округ (Архангел...  46.052632   9.868421   
                 Тюменская область (кроме Ханты-Манс...  54.568466  14.408801   
                 Ханты-Мансийский автономный округ -...  49.672270  14.402283   
                 Ямало-Ненецкий автономный округ (Тю...  45.403744  12.293937   

                                                                      second  \
woman_age                                                  40_100      18_20   
year region                                                                    
2018             Архангельская область (кроме Ненецк...  2.745952   6.344096   
                 Ненецкий автономный округ (Архангел...  1.644737  19.736842   
                 Тюменская область (кроме Ханты-Манс...  2.044492   7.983255   
                 Ханты-Мансийский автономный округ -...  2.229456   6.510010   
                 Ямало-Ненецкий автономный округ (Тю...  2.514669   9.918972   

                                                                    \
woman_age                                                    20_25   
year region                                                          
2018             Архангельская область (кроме Ненецк...  50.468706   
                 Ненецкий автономный округ (Архангел...  60.855263   
                 Тюменская область (кроме Ханты-Манс...  60.263837   
                 Ханты-Мансийский автономный округ -...  64.743390   
                 Ямало-Ненецкий автономный округ (Тю...  67.197541   

                                                                     \
woman_age                                                     25_30   
year region                                                           
2018             Архангельская область (кроме Ненецк...  162.011173   
                 Ненецкий автономный округ (Архангел...  141.447368   
                 Тюменская область (кроме Ханты-Манс...  157.717957   
                 Ханты-Мансийский автономный округ -...  155.125518   
              

In [26]:
new_df.sort_index(level=0, ascending=True).to_excel('test.xlsx') # сохраняю в эксель

In [27]:
# не нашел вменяемого способа избавиться от сложных мультииндексов, поэтому вот такой велосипед

new_df = pd.read_excel('test.xlsx', header=2) # когда читаем сохраненный файл - режутся индексы + сразу со второй строки прочел

new_df.year = sorted(i for _ in range(87) for i in (2018, 2019, 2020, 2021)) # переформировываю столбец с годами

# формирую список новых наименований столбцов
new_с = [f'{i}_{str(j).replace(", ", "_").replace("(", "").replace("]", "")}' for j in df.woman_age.unique() 
         for i in ('first', 'second', 'third', 'fourth', 'five_more')]

# меняю наименования столбцов
new_df = new_df.rename(columns={k: n for k, n in zip(new_df.columns[2:], new_с)}) 
new_df # смотрим, что получилось

,year,region,first_18_20,second_18_20,third_18_20,fourth_18_20,five_more_18_20,first_20_25,second_20_25,third_20_25,...,first_35_40,second_35_40,third_35_40,fourth_35_40,five_more_35_40,first_40_100,second_40_100,third_40_100,fourth_40_100,five_more_40_100
0,2018,Архангельская область (кроме Ненец...,38.443329,107.565571,125.366916,61.925954,16.665089,2.745952,6.344096,50.468706,...,8.616608,13.161632,12.120064,2.177824,0.000000,0.094688,3.882208,6.817536,7.006912,1.988448
1,2018,Ненецкий автономный округ (Арханге...,49.342105,93.750000,74.013158,46.052632,9.868421,1.644737,19.736842,60.855263,...,23.026316,37.828947,24.671053,4.934211,0.000000,0.000000,6.578947,19.736842,16.447368,0.000000
2,2018,Тюменская область (кроме Ханты-Ман...,38.942706,110.207857,123.545733,54.568466,14.408801,2.044492,7.983255,60.263837,...,12.656379,20.104172,13.581269,2.823346,0.000000,1.265638,6.279511,10.514530,8.956822,2.920703
3,2018,Ханты-Мансийский автономный округ ...,36.028002,122.620056,118.473269,49.672270,14.402283,2.229456,6.510010,64.743390,...,10.968921,20.466402,14.669818,2.541579,0.000000,0.401302,2.496990,6.911312,5.306104,2.006510
4,2018,Ямало-Ненецкий автономный округ (Т...,44.425817,111.902766,105.476390,45.403744,12.293937,2.514669,9.918972,67.197541,...,18.720313,23.609947,17.882090,2.654373,0.000000,0.558815,8.242526,17.183571,14.389494,3.632300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,2021,Челябинская область,43.313874,120.232298,101.751308,48.728109,16.696410,2.903723,9.255618,60.524485,...,12.764284,21.384713,16.545174,3.538913,0.000000,1.421615,6.049424,11.766129,9.981549,3.508666
344,2021,Чеченская Республика,72.209238,109.499588,38.424526,14.263129,5.395931,1.752818,23.267803,112.730272,...,69.769040,64.510586,21.033819,2.715150,0.068738,3.608743,37.255980,61.279901,33.131702,5.361562
345,2021,Чувашская Республика - Чувашия,33.163729,130.837725,94.039615,43.976013,13.628930,2.362348,5.633291,61.875341,...,9.631110,21.806287,18.353625,4.724696,0.000000,0.636017,4.179538,9.267672,10.085408,1.362893
346,2021,Чукотский автономный округ,87.934560,85.889571,75.664622,34.764826,12.269939,2.044990,28.629857,79.754601,...,16.359918,34.764826,12.269939,4.089980,0.000000,2.044990,10.224949,14.314928,8.179959,4.089980


In [28]:
# в результате получен датафрейм с детализацией по годам и регионам
# каждый столбец характеризует возможную комбинацию событий очередность рождения/возраст женщины
# думаю многие из указанных факторов неплохо будут коррелировать с индексом рождаемости первых/вторых/3 и более детей
# который и является, собственно, основным показателем рождаемости
new_df.info() # проверка на наличие пропусков - пропусков нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              348 non-null    int64  
 1   region            348 non-null    object 
 2   first_18_20       348 non-null    float64
 3   second_18_20      348 non-null    float64
 4   third_18_20       348 non-null    float64
 5   fourth_18_20      348 non-null    float64
 6   five_more_18_20   348 non-null    float64
 7   first_20_25       348 non-null    float64
 8   second_20_25      348 non-null    float64
 9   third_20_25       348 non-null    float64
 10  fourth_20_25      348 non-null    float64
 11  five_more_20_25   348 non-null    float64
 12  first_25_30       348 non-null    float64
 13  second_25_30      348 non-null    float64
 14  third_25_30       348 non-null    float64
 15  fourth_25_30      348 non-null    float64
 16  five_more_25_30   348 non-null    float64
 1

In [29]:
# функция для считки и присоединения новых данных
def merge_ver_1(df, join_df, new_col_names):
    # переименовываем столбцы
    join_df = join_df.rename(columns={i: n for i, n in zip(join_df.columns, new_col_names)})
    
    # исключаем строки с фед. округами и с РФ
    join_df = join_df[join_df['region'].apply(lambda x: 'федеральный' not in x and x != 'Российская Федерация')]

    # выявил методом проб и ошибок, расписывать не буду 
    join_df.region = join_df.region.apply(lambda x: x.replace('Архангельская область (без АО)', 
                                                              'Архангельская область (кроме Ненецкого автономного округа)') \
                                                     .replace('Тюменская область (без АО)', 
                                                              'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)')
                                         )
    return df.merge(join_df, how='left', on=['year', 'region']) # присоединяем к исходному датафрейму и возвращаем результат

In [150]:
# далее я просто по очереди читаю и присоединяю разные выгрузки данных
# первую выгрузку расписал вручную

In [30]:
# читаем файл с данными о числе зарегистрированных браков на 1000 населения
df_marr = pd.read_excel('число зарегистрированных браков на 1000 населения.xls', header=2)
df_marr.head()

,Unnamed: 0,Unnamed: 1,Число зарегистрированных браков в расчете на 1000 населения (оперативные данные)
0,2018,Российская Федерация,3.9
1,2018,Центральный федеральный округ,4.0
2,2018,Белгородская область,3.7
3,2018,Брянская область,3.5
4,2018,Владимирская область,3.6


In [31]:
# присоединяем
new_df = merge_ver_1(new_df, df_marr, ('year', 'region', 'married'))

In [32]:
new_df.isna().sum().sum() # проверяем на остутствие выпавших пропусков

0

In [33]:
new_df.head()

,year,region,first_18_20,second_18_20,third_18_20,fourth_18_20,five_more_18_20,first_20_25,second_20_25,third_20_25,...,second_35_40,third_35_40,fourth_35_40,five_more_35_40,first_40_100,second_40_100,third_40_100,fourth_40_100,five_more_40_100,married
0,2018,Архангельская область (кроме Ненец...,38.443329,107.565571,125.366916,61.925954,16.665089,2.745952,6.344096,50.468706,...,13.161632,12.120064,2.177824,0.0,0.094688,3.882208,6.817536,7.006912,1.988448,3.6
1,2018,Ненецкий автономный округ (Арханге...,49.342105,93.750000,74.013158,46.052632,9.868421,1.644737,19.736842,60.855263,...,37.828947,24.671053,4.934211,0.0,0.000000,6.578947,19.736842,16.447368,0.000000,1.3
2,2018,Тюменская область (кроме Ханты-Ман...,38.942706,110.207857,123.545733,54.568466,14.408801,2.044492,7.983255,60.263837,...,20.104172,13.581269,2.823346,0.0,1.265638,6.279511,10.514530,8.956822,2.920703,4.5
3,2018,Ханты-Мансийский автономный округ ...,36.028002,122.620056,118.473269,49.672270,14.402283,2.229456,6.510010,64.743390,...,20.466402,14.669818,2.541579,0.0,0.401302,2.496990,6.911312,5.306104,2.006510,5.6
4,2018,Ямало-Ненецкий автономный округ (Т...,44.425817,111.902766,105.476390,45.403744,12.293937,2.514669,9.918972,67.197541,...,23.609947,17.882090,2.654373,0.0,0.558815,8.242526,17.183571,14.389494,3.632300,5.4


In [155]:
# дальше я просто через цикл собрал все данные, удовлетворяющие условиям применения функции
# это все, кроме файлов "доля населения в АЖФ.xls" - там отличаются наименования регионов
# и файлов с численностью населения женщин и мужчин - из них можно взять более интересную информацию
# условные обозначения новых факторов:
# birth_1 - коэффициент рождаемости первых детей
# birth_2 - коэффициент рождаемости вторых детей
# birth_3 - коэффициент рождаемости третьих детей
# idx_price - базовый индекс потребительских цен
# idx_med - индекс цен на медицинские товары
# idx_school - индекс цен на одежду для детей школьного возраста
# idx_preschool - индекс цен на одежду для детей дошкольного возраста
# idx_baby - индекс цен на белье для детей ясельного возраста
# idx_school_up - индекс цен на услуги дошкольного воспитания
# per_zozh - доля населения ведущего зож
# divorce - число разводов на 1000 населения
# poverty_lvl - уровень бедности

In [34]:
# список наименований переменных
factors = (('idx_price', 'idx_med', 'idx_school', 'idx_preschool', 'idx_baby', 'idx_school_up'),
           ('per_zozh',), ('divorce',), ('poverty_lvl',),
           ('birth_1',), ('birth_2',), ('birth_3',))

# список наименований файлов
files = [i for i in ('индексы потребительских цен', 'доля населения ведущего зож',
                     'разводы на 1000 населения', 'уровень бедности',
                     'коэфф рождаемости', 'коэфф рождаемости вторых', 'коэфф рождаемости третьих+')]

for file, factor in zip(files, factors):
    new_df = merge_ver_1(new_df, pd.read_excel(f'{file}.xls', header=2), ('year', 'region', *factor))

In [35]:
# смотрим, что получилось
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 348 entries, 0 to 347
Data columns (total 45 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              348 non-null    int64  
 1   region            348 non-null    object 
 2   first_18_20       348 non-null    float64
 3   second_18_20      348 non-null    float64
 4   third_18_20       348 non-null    float64
 5   fourth_18_20      348 non-null    float64
 6   five_more_18_20   348 non-null    float64
 7   first_20_25       348 non-null    float64
 8   second_20_25      348 non-null    float64
 9   third_20_25       348 non-null    float64
 10  fourth_20_25      348 non-null    float64
 11  five_more_20_25   348 non-null    float64
 12  first_25_30       348 non-null    float64
 13  second_25_30      348 non-null    float64
 14  third_25_30       348 non-null    float64
 15  fourth_25_30      348 non-null    float64
 16  five_more_25_30   348 non-null    float64
 1

In [36]:
# увы, не по всем факторам удалось найти данные за 2018 год - усреднять значения по целому региону/году
# счел нецелесообразным, отбросить факторы тоже нельзя - один из них это индекс рождаемости первых детей,
# который нам как раз и нужно спрогнозировать, поэтому отбросил весь 2018 год
new_df = new_df.query('year > 2018')
new_df.isna().sum()[new_df.isna().sum().apply(lambda x: x > 0)] # ищем фактор с пропусками

per_zozh    2
dtype: int64

In [37]:
# собственно видим - проблема в том, что у нас нет данных по фактору "доля населения ведущего зож" за 2020 год
# в Ненецком автономном округе и за 2021 год в Чукотском автономном округе
new_df.query('per_zozh.isna()')

,year,region,first_18_20,second_18_20,third_18_20,fourth_18_20,five_more_18_20,first_20_25,second_20_25,third_20_25,...,idx_school,idx_preschool,idx_baby,idx_school_up,per_zozh,divorce,poverty_lvl,birth_1,birth_2,birth_3
175,2020,Ненецкий автономный округ (Арханге...,64.189189,103.040541,59.121622,30.405405,16.891892,1.689189,11.824324,57.432432,...,100.56,100.0,100.0,100.0,NaN,4.0,9.4,2.26,0.75,0.79
346,2021,Чукотский автономный округ,87.934560,85.889571,75.664622,34.764826,12.269939,2.044990,28.629857,79.754601,...,100.00,100.0,100.0,100.0,NaN,5.0,7.3,1.66,0.63,0.49


In [38]:
# можно взять средние значения в данных регионах за те годы, по которым у нас есть информация
nen = "            Ненецкий автономный округ (Архангельская область)"
new_df.loc[175, 'per_zozh'] = (int(new_df.query('year == 2019 and region == @nen').per_zozh) + 
                               int(new_df.query('year == 2021 and region == @nen').per_zozh)) / 2
chuk = "        Чукотский автономный округ"
new_df.loc[346, 'per_zozh'] = (int(new_df.query('year == 2019 and region == @chuk').per_zozh) + 
                               int(new_df.query('year == 2020 and region == @chuk').per_zozh)) / 2

In [39]:
# теперь соединим полученные результаты с данными о женском и мужском населении в возрасте от 17 до 35 лет
new_df = df_popul.merge(new_df, how='left', on=['year', 'region'])
new_df

,year,region,men_population_city_18_20,men_population_city_20_25,men_population_city_25_30,men_population_city_30_35,men_population_city_35_40,men_population_city_40_100,men_population_country_18_20,men_population_country_20_25,...,idx_school,idx_preschool,idx_baby,idx_school_up,per_zozh,divorce,poverty_lvl,birth_1,birth_2,birth_3
0,2019,Архангельская область (кроме Ненец...,857.692065,841.760469,833.786332,812.975801,799.733128,721.455984,142.307935,158.239531,...,99.90,99.93,99.61,100.37,13.5,4.4,12.7,1.46,0.57,0.32
1,2019,Ненецкий автономный округ (Арханге...,666.246851,727.195946,723.703704,795.857988,775.467775,677.752583,333.753149,272.804054,...,100.00,100.00,100.00,100.42,2.1,6.2,9.4,2.17,0.74,0.71
2,2019,Тюменская область (кроме Ханты-Ман...,655.934161,669.366424,760.707787,739.095143,710.483912,620.277318,344.065839,330.633576,...,100.00,100.03,100.05,100.00,16.9,4.5,14.4,1.75,0.62,0.43
3,2019,Ханты-Мансийский автономный округ ...,929.107478,920.409626,923.369748,925.402500,925.747283,914.996244,70.892522,79.590374,...,100.29,100.11,100.20,101.15,5.4,5.2,8.9,1.76,0.65,0.42
4,2019,Ямало-Ненецкий автономный округ (Т...,789.708585,799.581477,841.317365,868.065485,873.170917,855.890603,210.291415,200.418523,...,100.05,100.01,100.13,100.00,3.9,5.3,5.5,1.83,0.63,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,2021,Челябинская область,829.117968,797.103824,840.189987,841.228926,849.699354,808.672684,170.882032,202.896176,...,100.00,100.07,99.62,100.00,5.2,4.0,12.0,1.53,0.54,0.39
257,2021,Чеченская Республика,349.950364,354.663510,365.896759,422.155689,436.731398,363.960992,650.049636,645.336490,...,100.63,100.07,100.00,100.00,6.8,1.0,19.8,2.50,0.57,1.33
258,2021,Чувашская Республика - Чувашия,642.122761,557.749059,690.627202,662.072535,666.986521,541.795085,357.877239,442.250941,...,100.09,99.28,99.76,100.00,21.2,2.7,15.7,1.48,0.52,0.42
259,2021,Чукотский автономный округ,648.241206,806.693989,740.790355,753.108348,758.071107,737.025900,351.758794,193.306011,...,100.00,100.00,100.00,100.00,0.0,5.0,7.3,1.66,0.63,0.49


In [40]:
# видим, что пропусков у нас нет
new_df.isna().sum().sum()

0

In [41]:
# доля бедности (poverty_lvl), и доля населения ведущего зож (per_zozh) - измеряются в процентах, переведем в промилле
new_df.per_zozh = new_df.per_zozh * 10
new_df.poverty_lvl = new_df.poverty_lvl * 10

In [42]:
# divorce и married - число разводов и число заключенных браков на 1000 населения 
# это абсолютные показатели, тоже переведем в промилле (считаем внутри каждого года)

# считаем общую сумму заключенных и расторгнутых браков на 1000 населения за год и присоединяем к датафрейму
tmp = new_df.groupby('year', as_index=False) \
    .agg({'divorce': 'sum', 'married': 'sum'}) \
    .rename(columns={'divorce': 'divorce_full', 'married': 'married_full'})
new_df = new_df.merge(tmp, how='left', on='year')

# вычисляю долю расторгнутых браков в регионе от общего числа расторгнутых за год браков в РФ за год
new_df.divorce = (new_df.divorce / new_df.divorce_full * 1000).fillna(0)
# вычисляю долю заключенных браков в регионе от общего числа заключенных за год браков в РФ за год
new_df.married = (new_df.married / new_df.married_full * 1000).fillna(0)
# убираем столбцы с общими суммами
new_df.drop(['divorce_full', 'married_full'], axis= 1 , inplace= True)
new_df

,year,region,men_population_city_18_20,men_population_city_20_25,men_population_city_25_30,men_population_city_30_35,men_population_city_35_40,men_population_city_40_100,men_population_country_18_20,men_population_country_20_25,...,idx_school,idx_preschool,idx_baby,idx_school_up,per_zozh,divorce,poverty_lvl,birth_1,birth_2,birth_3
0,2019,Архангельская область (кроме Ненец...,857.692065,841.760469,833.786332,812.975801,799.733128,721.455984,142.307935,158.239531,...,99.90,99.93,99.61,100.37,135.0,12.621916,127.0,1.46,0.57,0.32
1,2019,Ненецкий автономный округ (Арханге...,666.246851,727.195946,723.703704,795.857988,775.467775,677.752583,333.753149,272.804054,...,100.00,100.00,100.00,100.42,21.0,17.785427,94.0,2.17,0.74,0.71
2,2019,Тюменская область (кроме Ханты-Ман...,655.934161,669.366424,760.707787,739.095143,710.483912,620.277318,344.065839,330.633576,...,100.00,100.03,100.05,100.00,169.0,12.908778,144.0,1.75,0.62,0.43
3,2019,Ханты-Мансийский автономный округ ...,929.107478,920.409626,923.369748,925.402500,925.747283,914.996244,70.892522,79.590374,...,100.29,100.11,100.20,101.15,54.0,14.916810,89.0,1.76,0.65,0.42
4,2019,Ямало-Ненецкий автономный округ (Т...,789.708585,799.581477,841.317365,868.065485,873.170917,855.890603,210.291415,200.418523,...,100.05,100.01,100.13,100.00,39.0,15.203672,55.0,1.83,0.63,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,2021,Челябинская область,829.117968,797.103824,840.189987,841.228926,849.699354,808.672684,170.882032,202.896176,...,100.00,100.07,99.62,100.00,52.0,13.422819,120.0,1.53,0.54,0.39
257,2021,Чеченская Республика,349.950364,354.663510,365.896759,422.155689,436.731398,363.960992,650.049636,645.336490,...,100.63,100.07,100.00,100.00,68.0,3.355705,198.0,2.50,0.57,1.33
258,2021,Чувашская Республика - Чувашия,642.122761,557.749059,690.627202,662.072535,666.986521,541.795085,357.877239,442.250941,...,100.09,99.28,99.76,100.00,212.0,9.060403,157.0,1.48,0.52,0.42
259,2021,Чукотский автономный округ,648.241206,806.693989,740.790355,753.108348,758.071107,737.025900,351.758794,193.306011,...,100.00,100.00,100.00,100.00,0.0,16.778523,73.0,1.66,0.63,0.49


In [43]:
# пересчет индексных переменных в промилле
# индексы за 2019 год возьмем за исходные значения, т.е. значения 2019 года равны 1000 промилле
# а индексы следующего года будем считать, как промилле от индексов предыдущего года
ind_new = pd.DataFrame()

cols = ['birth_1', 'birth_2', 'birth_3', 'idx_price', 'idx_med', 
        'idx_school', 'idx_preschool', 'idx_baby', 'idx_school_up']

for y in (2019, 2020, 2021):
    # первый год берем как исходный
    if y == 2019:
        temp = new_df.query('year == @y')[cols].reset_index(drop=True) \
        / new_df.query('year == @y')[cols].reset_index(drop=True) * 1000
    else:
        temp = new_df.query('year == @y')[cols].reset_index(drop=True) \
        / new_df.query('year == @y - 1')[cols].reset_index(drop=True) * 1000
    ind_new = pd.concat([ind_new, temp], axis=0)

new_df[cols] = ind_new.reset_index(drop=True)

In [44]:
new_df.head()

,year,region,men_population_city_18_20,men_population_city_20_25,men_population_city_25_30,men_population_city_30_35,men_population_city_35_40,men_population_city_40_100,men_population_country_18_20,men_population_country_20_25,...,idx_school,idx_preschool,idx_baby,idx_school_up,per_zozh,divorce,poverty_lvl,birth_1,birth_2,birth_3
0,2019,Архангельская область (кроме Ненец...,857.692065,841.760469,833.786332,812.975801,799.733128,721.455984,142.307935,158.239531,...,1000.0,1000.0,1000.0,1000.0,135.0,12.621916,127.0,1000.0,1000.0,1000.0
1,2019,Ненецкий автономный округ (Арханге...,666.246851,727.195946,723.703704,795.857988,775.467775,677.752583,333.753149,272.804054,...,1000.0,1000.0,1000.0,1000.0,21.0,17.785427,94.0,1000.0,1000.0,1000.0
2,2019,Тюменская область (кроме Ханты-Ман...,655.934161,669.366424,760.707787,739.095143,710.483912,620.277318,344.065839,330.633576,...,1000.0,1000.0,1000.0,1000.0,169.0,12.908778,144.0,1000.0,1000.0,1000.0
3,2019,Ханты-Мансийский автономный округ ...,929.107478,920.409626,923.369748,925.402500,925.747283,914.996244,70.892522,79.590374,...,1000.0,1000.0,1000.0,1000.0,54.0,14.916810,89.0,1000.0,1000.0,1000.0
4,2019,Ямало-Ненецкий автономный округ (Т...,789.708585,799.581477,841.317365,868.065485,873.170917,855.890603,210.291415,200.418523,...,1000.0,1000.0,1000.0,1000.0,39.0,15.203672,55.0,1000.0,1000.0,1000.0


In [45]:
# отбрасываем столбцы с годом и регионом и сохраняем в csv файл
# датасет готов.
final_dataset = new_df.drop(['year', 'region'], axis=1)
final_dataset.to_csv('final_dataset.csv')

In [46]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261 entries, 0 to 260
Data columns (total 67 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   men_population_city_18_20        261 non-null    float64
 1   men_population_city_20_25        261 non-null    float64
 2   men_population_city_25_30        261 non-null    float64
 3   men_population_city_30_35        261 non-null    float64
 4   men_population_city_35_40        261 non-null    float64
 5   men_population_city_40_100       261 non-null    float64
 6   men_population_country_18_20     261 non-null    float64
 7   men_population_country_20_25     261 non-null    float64
 8   men_population_country_25_30     261 non-null    float64
 9   men_population_country_30_35     261 non-null    float64
 10  men_population_country_35_40     261 non-null    float64
 11  men_population_country_40_100    261 non-null    float64
 12  women_population_city_